# Churn analysis

Here we will try and identify variables that correlate with whether a user churns or not. 

To do this, we'll first load and transform the data into a format that will make running the correlations easy. Given a mix of continuous and discrete variables, we'll need to separately run pearson and chi2 correlations to detemine the significance of the variables for churn. Then we'll plot the data, to allow visual confirm any correlation we might have identified. Finally, we'll built a model to try and predict whether users will churn or not, using the scikit-learn package.

**Data**

USERS.json: A list of users containing some descriptive data (age/height/weight), as well as their answers to the sign-up quiz (motivation/challenge level).

EVENTS.json: A list of all the events logged by users (articles read, weigh-ins, etc)

MESSAGES.json: A log of all messages sent by users. This does not include message content, but rather a sentiment score and other features to determine whether questions were asked, or peripherals were mentioned. 


**Notes**

Where plots or csv files are generated in this script, I've commented out the lines that actually write the files. This is to avoid accident overwriting of the figures, if you choose to edit the script yourself. I've included the figures in the repo (in the 'figs' folder), so you browse them there. Else, if you want to uncomment the lines and save the plots yourself, feel free to!

First we'll load our dependencies.

In [1]:
### Load Dependencies

import os

# For data manipulation 
import numpy as np
import pandas as pd

# For stats
import scipy.stats as spstats

# For modeling
import sklearn.metrics as skm
from sklearn.model_selection import train_test_split 
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

# For plotting
import matplotlib.pyplot as plt
from utils import GetTransform, restyle

plt.style.use('./ag_light.mplstyle')

Also, we can define a quick function that'll help us out later, when it comes to using multi-level columns in pandas.

In [2]:
def flatten_columns(columns):
    """Flatten a multilevel column set into '_' separated column names"""

    columns_joined = []
    for levels in columns.to_list():
        levels = [str(x) for x in levels if x != '']    
        columns_joined.append("_".join(levels))

    return columns_joined

## Loading the data

We can import the three JSON files  using pandas' built-in read_json function.

In [3]:
# Load json files
msg = pd.read_json('./json/MESSAGES.json')
evnt = pd.read_json('./json/EVENTS.json')
usr = pd.read_json('./json/USERS.json')

Now we need to clean up the dataframes - notably the column names. They'll be less human-friendly, but more code-friendly.

In [4]:
# Unpack id fields from dict's to strings
func = lambda x: x['$oid']
msg[['_id','user']] = msg[['_id','user']].applymap(func)
evnt[['_id','user']] = evnt[['_id','user']].applymap(func)
usr['_id'] = usr['_id'].apply(func)

usr.rename(columns={'_id':'user'}, inplace=True)

# Fix column names in events dataframe
code_title = {
    'Added new food diary entry': 'diary',
    'Sent message': 'message',
    'Read article': 'article',
    'Weigh-in': 'weight',
    'Saved recipe': 'recipe'
}
evnt['title'] = evnt['title'].apply(lambda x: code_title.get(x,x))
evnt.rename(columns={'weekOnProgramme':'weekNumber'}, inplace=True)

Next, we'll set up the primary dataframe to house our data. To initialise it, we just make a row for each week, for each user

In [5]:
data_weekly = usr['user'].repeat(7).reset_index(drop=True).to_frame()
data_weekly = pd.merge(data_weekly, usr[['user','churnedAfterSix']], on='user')
data_weekly['weekNumber'] = np.tile(range(7), usr['user'].nunique())

print(data_weekly)

                          user churnedAfterSix  weekNumber
0     5a2e417806d240124a6185a0              NA           0
1     5a2e417806d240124a6185a0              NA           1
2     5a2e417806d240124a6185a0              NA           2
3     5a2e417806d240124a6185a0              NA           3
4     5a2e417806d240124a6185a0              NA           4
...                        ...             ...         ...
3061  5ce6663e7de10312a89e572b           False           2
3062  5ce6663e7de10312a89e572b           False           3
3063  5ce6663e7de10312a89e572b           False           4
3064  5ce6663e7de10312a89e572b           False           5
3065  5ce6663e7de10312a89e572b           False           6

[3066 rows x 3 columns]


## Data prep (events data)

We can start to populate this dataframe by transforming the data from EVENTS.json. Here, for each user we want to know how many of each event they performed, per week. We'll impute missing values with 0, as a missing entry implies they did not do the event that week. 

In [6]:
# add 'Number of each event, by week'
data_events = (
    evnt.groupby(['user','weekNumber','title'])
    .count()
    .unstack(['title'])
)
data_events.columns = data_events.columns.droplevel(0)
data_events = pd.merge(data_weekly[['user','weekNumber']], data_events, on=['user','weekNumber'], how='left')

# Fill missing values with 0
tf = data_events.columns.str.contains('arti|diar|mess|reci|weig')
data_events.loc[:,tf] = data_events.loc[:,tf].fillna(0)

We can also generate an additional two features. First is the *total number* of events logged, per user, per week. Though, we don't include 'messages sent' in this, as we will perform separate analyses on the message dataset in a sec.

The second is the 'change in no. events', by week. Essentially, this indicates if the user is doing more or fewer events, week on week. We'll add the prefix 'delta_' to these features, as the delta symbol ($\Delta$) is commonly used to denote 'change of' a given value.

In [7]:
# Add 'Change in...' metrics
tf = data_events.columns.str.contains('arti|diar|reci|weig')
data_events['numberEvents'] = data_events.loc[:,tf].sum(1)
data_events['delta_numberEvents'] = data_events.groupby('user')['numberEvents'].diff().fillna(0)

data_events.head()

,user,weekNumber,article,diary,message,recipe,weight,numberEvents,delta_numberEvents
0,5a2e417806d240124a6185a0,0,0.0,1.0,3.0,0.0,0.0,1.0,0.0
1,5a2e417806d240124a6185a0,1,5.0,1.0,0.0,0.0,2.0,8.0,7.0
2,5a2e417806d240124a6185a0,2,0.0,0.0,0.0,0.0,1.0,1.0,-7.0
3,5a2e417806d240124a6185a0,3,4.0,0.0,2.0,0.0,2.0,6.0,5.0
4,5a2e417806d240124a6185a0,4,0.0,0.0,0.0,0.0,1.0,1.0,-5.0


This can now be appended to our main dataframe

In [8]:
# Append to main dataframe
data_weekly = pd.merge(data_weekly, data_events, on=['user','weekNumber'], how='left')
del(data_events)

## Data prep (message data)

For the message dataset we want to calculate the average usage for each user, per week, and per messageType (group or private message).

In [9]:
# Calculated average metrics, per user, per week
msg_weekly = (
    msg.groupby(['user','weekNumber','messageType'])
    .agg(
        messageCount = ('sentiment','count'),
        sentiment = ('sentiment', 'mean'),
        questionsAsked = ('questionsAsked', 'sum'),
        emojisUsed = ('emojisUsed', 'sum'),
        mentionedScales = ('mentionedScales', lambda x: int(any(x>0))),
        mentionedTracker = ('mentionedTracker', lambda x: int(any(x>0))),
    )
    .unstack(['messageType'])
    .reset_index()
)
msg_weekly.columns = flatten_columns(msg_weekly.columns)

# Expand rows to include weeks where users have no messages logged
msg_weekly = pd.merge(
    data_weekly[['user','weekNumber','churnedAfterSix']],
    msg_weekly, 
    on=['user','weekNumber'], 
    how='left'
)

# Impute missing values with 0
tf = msg_weekly.columns.str.contains('mess|quest|mention|emoji')
msg_weekly.loc[:,tf] = msg_weekly.loc[:,tf].fillna(0)

We can engineer more 'change in x' features at this point, for sentiment, messageCount, questionAsked, mentionedTracker and mentionedScales.

In [10]:
### Engineer additional features (message data)

# Calculate change in sentiment & no. messages
tf = msg_weekly.columns.str.contains('^senti|^messa|^quest|^emoji|^menti')
delta_sentiment = (
    msg_weekly.loc[:,tf]
    .groupby(msg_weekly['user'])
    .diff(axis=0)
    .fillna(0)
    .add_prefix('delta_')
)
msg_weekly = pd.concat((msg_weekly,delta_sentiment), axis=1)
msg_weekly.drop(columns=['churnedAfterSix'], inplace=True)

Again, we add append this to our main dataframe, and save a copy of the dataframe (in case we want to peruse it at a later point)

In [11]:
# Append to main dataframe
data_weekly = pd.merge(data_weekly, msg_weekly, how='left', on=['user','weekNumber'])
del(msg_weekly)

# Save reformatted data
# data_weekly.round(3).to_csv(os.path.join('csv','data_user_weekly.csv'))
print(data_weekly)

                          user churnedAfterSix  weekNumber  article  diary  \
0     5a2e417806d240124a6185a0              NA           0      0.0    1.0   
1     5a2e417806d240124a6185a0              NA           1      5.0    1.0   
2     5a2e417806d240124a6185a0              NA           2      0.0    0.0   
3     5a2e417806d240124a6185a0              NA           3      4.0    0.0   
4     5a2e417806d240124a6185a0              NA           4      0.0    0.0   
...                        ...             ...         ...      ...    ...   
3061  5ce6663e7de10312a89e572b           False           2      6.0    6.0   
3062  5ce6663e7de10312a89e572b           False           3      6.0    0.0   
3063  5ce6663e7de10312a89e572b           False           4      0.0    0.0   
3064  5ce6663e7de10312a89e572b           False           5     10.0    0.0   
3065  5ce6663e7de10312a89e572b           False           6      0.0    0.0   

      message  recipe  weight  numberEvents  delta_numberEvents

## Format data for model

The last thing we're going to do before running any correlations is unstack the dataframe, so that each column contains a variable for a specific week.

In [12]:
# Format data
data_model = data_weekly.pivot(index=['user','churnedAfterSix'],columns='weekNumber').reset_index('churnedAfterSix')
data_user = usr[['user','age','height','weight']].set_index('user')

col_names = list(zip(*[np.repeat('userData', data_user.shape[1]),data_user.columns]))
data_user.columns = pd.MultiIndex.from_tuples(col_names)

data_model = pd.merge(data_user,data_model, on='user').set_index('churnedAfterSix',append=True)
print(data_model.shape)

(438, 220)


Given that we're analysing each metric at each time point, and additional generated features for each metric, we've now got 220 features that can be analysed. In hindsight, this was probably overkill for the task at hand, but offered the greatest breadth of metrics for our model.


## Correlation analysis (continuous data)

With all our continuous variables arranged, we'll now run a pearsons correlation on each of the variables to determine whether they correlate significantly with incidence of churning. This is also commonly referred to as a point bisceral correlation, but for all intents and purposes, it's a pearsons r calculation.

We'll print those results with the greatest significance, and save all the stats to a csv file.

In [13]:
# Remove the data where churn is not known
data_pearson = data_model.query("churnedAfterSix != 'NA'")

stats = []
# loop through each column in the dataframe
for col in data_pearson.columns.to_list():

    # x will be binary (0 or 1), y will be continuous
    x = data_pearson.index.get_level_values('churnedAfterSix').to_numpy(int)
    y = data_pearson[col].to_numpy()
    
    # filter out values where there are nan's - only relevant for sentiment column
    tf = ~np.isnan(y)
    x,y = x[tf],y[tf]
    
    # calculate a couple of additional stats
    meanFalse = y[x==0].mean()
    meanTrue = y[x==1].mean()
    nFalse = len(y[x==0])
    nTrue = len(y[x==1])
    
    # run the correlation
    r,p = spstats.pearsonr(x,y)
    
    # append results to list of stats
    res = [col[1], col[0], nFalse, meanFalse, nTrue, meanTrue, r, p]
    stats.append(res)
        
# Create, format and sort stats dataframe     
stats = np.asarray(stats)
dfStats = pd.DataFrame(data=stats, columns=['week', 'features', 'nFalse', 'aveFalse', 'nTrue', 'aveTrue', 'pearsonsR', 'pval'])
dfStats.iloc[:,2:] = dfStats.iloc[:,2:].astype(float).round(3)
dfStats = dfStats.sort_values(by=['pval'], ascending=True)

# Save summary stats
dfStats = dfStats.dropna(axis=0)
# dfStats.to_csv(os.path.join('csv','stats_pearsonr_group_weekly.csv'))

# Print results
dfStats.head(10)

,week,features,nFalse,aveFalse,nTrue,aveTrue,pearsonsR,pval
3,0,article,390.0,4.346,38.0,2.526,-0.119,0.013
154,4,delta_sentiment_group,390.0,-0.003,38.0,0.041,0.106,0.028
75,2,sentiment_private,232.0,0.153,23.0,0.23,0.136,0.03
61,2,messageCount_private,390.0,2.128,38.0,3.342,0.102,0.035
93,6,questionsAsked_private,390.0,0.103,38.0,0.316,0.101,0.036
110,2,mentionedScales_group,390.0,0.103,38.0,0.0,-0.1,0.038
144,1,delta_messageCount_private,390.0,1.041,38.0,2.184,0.094,0.053
208,2,delta_mentionedTracker_group,390.0,-0.023,38.0,0.053,0.093,0.054
188,3,delta_emojisUsed_private,390.0,-0.177,38.0,-0.553,-0.093,0.056
146,3,delta_messageCount_private,390.0,-0.744,38.0,-1.947,-0.092,0.057


In [14]:
dfStats.query("pearsonsR > 0.1 | pearsonsR < -0.1")


,week,features,nFalse,aveFalse,nTrue,aveTrue,pearsonsR,pval
3,0,article,390.0,4.346,38.0,2.526,-0.119,0.013
154,4,delta_sentiment_group,390.0,-0.003,38.0,0.041,0.106,0.028
75,2,sentiment_private,232.0,0.153,23.0,0.23,0.136,0.03
61,2,messageCount_private,390.0,2.128,38.0,3.342,0.102,0.035
93,6,questionsAsked_private,390.0,0.103,38.0,0.316,0.101,0.036
79,6,sentiment_private,102.0,0.145,8.0,0.053,-0.14,0.144
78,5,sentiment_private,81.0,0.133,11.0,0.188,0.114,0.281


Given that we're looking at over 200 features, we would typically look to set a strict significance threshold (alpha value), above which we would discard features. This would reduce the chance that one of our significant correlations occured simply by chance. Unfortunately, our smallest p value is $p = 0.013$, so we should not outright conclude that any of our correlations are statistically significant (typical alpha value might be 0.01, or 0.001). 

However, given that we're trying to build a model to predict churn, it is less important that the p values lie below a certain threshold. Ultimately, our model will take in numerous features as input, so it's likely that the interaction between features will be more important than their standalone correlation with churn. 

Some features crop up more than others in this list, including the message count and number of questions asked in private message. These features may well show covariance, as if you have more questions, you'll likely send more messages. 

The variable that gives the most significant correlation is the number of articles a user reads prior to starting the programme (week 0).

Interestingly, the strongest correlations (pearson's R value, rather than p value), arise from sentiment in private messages. These are less significant, because there are a number of missing values in the dataset (users didn't necessary log a message every week). However they may still be interesting to look at.

## Visualising continuous data

Now we'll plot each variable as a timeseries, to see how the variables change over the course of the six week period.

First we need to set up some parameters for the plots.

In [15]:
# Calculate mean weekly values
data_mean = data_model.groupby('churnedAfterSix').mean()
data_mean.round(3).T.to_csv(os.path.join('csv','data_average_weekly.csv'))

# Adjust y scale for mentionedScales and mentionedTracker
tf = data_mean.columns.get_level_values(0).str.contains('^mention')
data_mean.loc[:,tf] = data_mean.loc[:,tf]*100

# Set up plot parameters
titles = {
    'messageCount': "Average no. messages sent, per user",
    'sentiment': "Average sentiment, per user",
    'questionsAsked': "Average no. questions asked, per user",
    'mentionedScales': "% of users that mentioned 'scales'",
    'mentionedTracker': "% of users that mentioned 'tracker'",
    'emojisUsed': "Average no. times a user used an emoji",
    'article': "no. Articles read, per user",
    'diary': "no. Food diary entries added, per user",
    'weight': "no. Weigh-ins, per user",
    'recipe': "no. Recipes saved, per user",
    'numberEvents': "no. Events logged, per user"
}

colors = {
    False: 'black',
    True: (1,0,0)
}

limits = {
    'sentiment': [0, 0.25],
}

labels = {
    True: 'Churned',
    False: 'Completed'
}

Then we can generate the plots.

In [16]:
plt.rcParams.update({'font.size': 6})
fig = plt.figure(figsize=(3,2))

for col in data_mean.columns.get_level_values(0).unique().to_list():
    # Skip column if not in 'titles'
    col_parts = col.split('_')
    if col_parts[0] not in titles.keys():
        continue
    
    # Format data
    data = data_mean.query("churnedAfterSix != 'NA'")[col].T#.unstack('churnedAfterSix')
    
    # Unpack variables to plot
    x = data.index.to_numpy() * [[1],[1]]
    y = data.to_numpy().T
    label = [labels[c] for c in data.columns]
    color = [colors[c] for c in data.columns]
    
    # Plot the lines
    ax = fig.add_axes([0.15,0.2,0.7,0.6])
    ax.plot(x[0], y[0], c=color[0], lw=2)
    ax.plot(x[1], y[1], c=color[1], lw=2)
    
    # Tidy up plot
    restyle(ax)
    ax.set_xticks(np.arange(0,7,2))
    ax.set_ylim(ymin=0)
    if col_parts[0] in limits.keys():
        ax.set_ylim(limits[col_parts[0]])
    
    # Add labels
    trans = GetTransform(ax,system='8pt', anchor='tl')
    ax.text(0,2,titles[col_parts[0]], size=8, weight='bold', transform=trans)
    if col_parts[-1] in ['private', 'group']:
        ax.text(0,1,f'{col_parts[-1].title()} messages', size=6, transform=trans)
    
    ax.set_xlabel('Week')
    
    # Add legend
    trans = GetTransform(ax, system='8pt', anchor='tr')
    ax.text(0,-1, label[False], c=colors[False], size=6, weight='bold', ha='right', transform=trans)
    ax.text(0,-2, label[True], c=colors[True], size=6, weight='bold', ha='right', transform=trans)
    
    # Save plot
    filename = os.path.join('.','figs',f'{col}.png')
    # fig.savefig(filename, dpi=450)
    # fig.savefig(filename.replace('.png','.svg'))
    fig.clear()

plt.close()

Below are an example of how these plots look.

![](figs/article.svg)
![](figs/messageCount_private.svg)

## Correlation analysis (discrete data)

Now we can assess the correlation between our discrete variables and churn. For example, we can ask whether identifying as male or female significantly changes the likelihood that a user will churn. To do this we'll use the chi-squared test, which takes a contingency as input, and returns the correlation and p values associated with that table.

In [17]:
# Format data
dummy_cols = ['goalsAspiration', 'gender', 'motivation', 'challenge', 'trigger']
dummies = pd.get_dummies(usr[dummy_cols], dummy_na=True) # In this instance, we dont want to 'drop_first'
data_quiz = dummies.groupby(usr.churnedAfterSix).agg(['sum',lambda x: x.shape[0]])
data_quiz = data_quiz.query("churnedAfterSix != 'NA'")

chi = []; pval = []; index = []
for col in set(data_quiz.columns.get_level_values(0)):
    try:
        c,p,*_ = spstats.chi2_contingency(data_quiz[col])
        # c,p = spstats.fisher_exact(data_quiz[col])
    except:
        continue

    chi.append(round(c,3))
    pval.append(round(p,3))
    index.append(col)
    
chi2 = pd.DataFrame(data=np.asarray([chi,pval]).T, index=index, columns=['chi2','pval'])
chi2.sort_values('pval', inplace=True)
# chi2.to_csv(os.path.join('csv','stats_chi2_categorical.csv'))

chi2.head()


,chi2,pval
motivation_other,1.275,0.259
challenge_time,0.644,0.422
trigger_routine,0.624,0.430
trigger_emotions,0.547,0.459
gender_M,0.477,0.490


The most significant value returned was $p = 0.259$, suggesting there is little correlation of any of our variables with churn. Still, we can plot the data, as a sanity check, to see if any trends are visually apparent. 

First we'll define some plot parameters.

In [18]:
# Reformat dataframe
data_perc = data_quiz.groupby(level=0, axis=1).apply(lambda x: x.iloc[:,0] / x.iloc[:,1]).T
index = np.array([string.split('_') for string in data_perc.index])

colors = {
    False: 'black',
    True: (1,0,0)
}

legend = {
    True: 'Churned',
    False: 'Completed'
}

plt.rcParams.update({
    'font.size': 6,
    'axes.grid.axis': 'x',
    'xtick.bottom': False,
    'ytick.labelleft': False,
    'axes.ymargin': .1,
})

Then we'll generate the plots.

In [19]:
# Define the groups (column fields)
groups = set(index[:,0])
fig = plt.figure(figsize=(2,3))

for grp in groups:
    # Select the appropriate columns & y labels
    tf = data_perc.index.str.contains(f'^{grp}')
    labels = data_perc[tf].index.to_numpy()
    labels = [lab.split('_')[-1] for lab in labels]
    
    # Pull the x and y data
    xl,xr = data_perc.loc[tf].to_numpy().T * [[-1],[1]]
    y = np.arange(len(xl))
    
    # Draw the boxes
    ax = fig.add_axes([0.2,0.2,0.5,0.6])
    ax.barh(y, xl, 0.5, fc=colors[False])
    ax.barh(y, xr, 0.5, fc=colors[True])
    
    # Clean up axes
    ax.invert_yaxis()
    ax.set_xlim([-1,1])
    ax.set_xticklabels([100,50,0,50,100])
    ax.set_xlabel("% of users")
    # Draw y labels
    for pos,lab in zip(y,labels):
        ax.text(1.05, pos, lab, va='center', transform=ax.transData)
    
    # Add title
    trans = GetTransform(ax,system='10pt', anchor='tl')
    ax.text(0,3, f"% of responses to", size=8, weight='bold', transform=trans)
    ax.text(0,2, f"'{grp}' question", size=8, weight='bold', transform=trans)
    
    # Draw legend
    trans = GetTransform(ax, system=('data','6pt'), anchor='bl')
    ax.text(-1, 0, legend[False], c=colors[False], size=6, weight='bold', transform=trans)
    ax.text(1, 0, legend[True], c=colors[True], size=6, weight='bold', ha='right', transform=trans)
    
    # Save figure
    filename = os.path.join('.','figs',f'{grp}.png')
    # fig.savefig(filename, dpi=450)
    # fig.savefig(filename.replace('.png','.svg'), dpi=450)
    fig.clear()
    
plt.close()

The plots look like those shown below.

They show the percentage of users that chose each response to a given sign-up quiz question, split by users that churned (red) and those that completed (black) the programme. While there are some discernible differences visually ('motivation' response to 'challenge' question, or 'social' response to the 'trigger' question), the differences are not so great as to ignore the statistical results from the chi-squared test.

![](figs/motivation.svg)
![](figs/trigger.svg)
![](figs/challenge.svg)


## Modeling churn

Now we're going to try and build a model that can predict whether users will churn or not. To do this, we choose a handful of features that correlate well with churn, then use these to train the model. For this this task I've chosen to use a Random Forest Classification model, predominantly beacuse I haven't used one before and I wanted to learn more about them.

First we need to impute the missing values for our sentiment data (the model won't accept nan's). I've decided to impute a user's missing values with the same user's mean, as it seems like most conservative approach to handling these values. Impute with 0's is another option, but this would have skewed the user's mean sentiment.


In [20]:
# Impute user with incomplete data
fill_mean = lambda x: x.fillna(x.mean())
cols = ['sentiment_private', 'sentiment_group']
pivot = (
    data_model.loc[:,cols]
    .stack(level=1, dropna=False)
    .groupby(level=0)
    .transform(fill_mean)
)

# Impute users with no data at all
data_model.loc[:,cols] = (
    pivot.transform(fill_mean)
    .unstack(level=-1)
)
data_model.reset_index(inplace=True)
data_model.set_index(['user','churnedAfterSix'], inplace=True)

Next we'll select the features with which to train the model and set up the training, test and validation datasets. I've opted to start with a simpler model, so just picked the 8 features with the greatest correlation score.

In [21]:
# Set how many features we want to include in the model
nfeatures = 8

# Add absolute correlation, to help sorting
dfStats['pearsonsRAbs'] = dfStats['pearsonsR'].abs()

# Get the keys (names) for the features
keys = (
    dfStats.sort_values('pearsonsRAbs', ascending=False)
    .loc[:,['week', 'features']][:nfeatures]
    .to_numpy()[:,::-1]
)
keys[:,1] = keys[:,1].astype(int)

# Set up the label array (churned or not)
churned = data_model.index.get_level_values('churnedAfterSix')
tf = churned != 'NA'
y = np.array([int(v) for v in churned[tf]])

# Normalise the data
X_raw = data_model[keys].to_numpy()
X = (X_raw - X_raw.mean(0)) / X_raw.std(0)

# Set up the train, test and validate sections of the data
X_valid = X[~tf,:]
X_train,X_test,y_train,y_test = train_test_split(X[tf,:], y, test_size=0.25, random_state=0)

Finally, we can define, train and test the model. Again, I've opted for a simpler model as a starting point, defining the model only use 100 trees and a maximum branch depth of 5. This leaves room for increasing complexity while tinkering/optimising the model.

In [22]:
# Define the model
clf = RandomForestClassifier(
    n_estimators=100,
    random_state=0,
    max_depth=5,
    class_weight={0:1, 1:8}
)

# Train the model with the training dataset
clf.fit(X_train,y_train)

# Test the accuracy on the training dataset
y_pred = clf.predict(X_train)
# skmetrics.confusion_matrix(y_train,y_pred)s

# Test the accuracy on the test dataset
y_pred = clf.predict(X_test)
print(skm.confusion_matrix(y_test,y_pred))

# Calculate model metrics
accuracy = skm.accuracy_score(y_test, y_pred)
precision = skm.precision_score(y_test, y_pred)
recall = skm.recall_score(y_test, y_pred)

string = f"""
    Model results:
    \t accuracy  = {accuracy:.3f}
    \t recall    = {recall:.3f}
    \t precision = {precision:.3f}
"""
print(string)

[[95  5]
 [ 5  2]]

    Model results:
    	 accuracy  = 0.907
    	 recall    = 0.286
    	 precision = 0.286



The accuracy of this model is defined as the number of users it correctly classifies as either churner or completer. While the model reveals 90%, this is misleading, as really we're interested in the number of churners, specifically, that we're accurately identifying. For this we look at the model 'precision', which paints a sorrier picture, with only 29% precision (2 out of 7 churners correctly identified).

Similarly, we incorrectly identified 5 individuals as churners, when they did not actually churn. This is measured as our 'recall' rate, at 29%.

Despite the fact the the precision and recall of this model are far from optimal, this is a decent starting point for further optimisation of the model. For one, we could choose to include more features into the model, as we've been fairly conservative in selecting 8. We could also increase the depth of the trees, which is likely to improve the recall, but at the cost of precision, as the model may start to overfit to the training data. Additionally, we could adjust the weighting of the class (churn vs not churn) to encourage the model to identify more churners, at the risk of identifying more false positives.

Last but not least, we can make our predictions for the unknown data:

In [23]:
y_pred = clf.predict(X_valid)
predictions = usr.query("churnedAfterSix == 'NA'")[['user']]
predictions['willChurn'] = y_pred.astype(bool)
print(predictions)

                         user  willChurn
0    5a2e417806d240124a6185a0      False
44   5ca8f158540a036b663a34df       True
55   5cba40489203684bd4c4e04f      False
58   5cbc31e194b8d6115731aed9      False
72   5cc929eba4cedc162e6df3c2      False
134  5cd520832925a912cbdd6061      False
178  5cd69b69fb622f12c5a74008      False
230  5cd89558f2f52212d094bd8e       True
240  5cd976930b932012b968558e      False
364  5ce050278ff304131108e204      False
